In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Подключите необходимые библиотеки.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(10, 8)}); # you can change this if needed

Загрузите данные. Они находятся в директории input.

In [ ]:
df = pd.read_csv('../input/bigml_59c28831336c6604c800002a.csv', sep = ',')
df.head(6)

#### *Целевая переменная(taget) -- churn, что в переводе означает "отток". Грубо говоря "отток" означает, что клиент перестаёт пользоваться услугами компании. Как мы видим, данная переменная принимает два значения: True  и False. То есть, True - означает, что клиент перестал пользоваться услугами компании, False - клиент продолжает пользоваться услугами. Основная задача состоит в том, чтобы уменьшить отток пользователей, проанализировать причины, по которым клиенты перестали пользоваться услугами компании,и предотвратить уход клиентов в дальнейшем, а также повысить удовлетворенность клиентов.*

In [ ]:
df.info()

Каково распределение абонентов по целевому признаку **churn**? Назовите процент лояльных абонентов.

In [ ]:
df['churn'].value_counts().plot(kind = 'bar', color = 'orange');

In [ ]:
# Your code
df['churn'].value_counts(normalize = True)

## **Процент лояльных абонентов ~ 85,5%**

Жители какого штата говорят по телефону больше всего? меньше всего? Обратите внимание, что нас интересуют все виды звонков.

In [ ]:
df.head(6)

In [ ]:
df['totalMinutes'] = df['total day minutes'] + df['total eve minutes'] + df['total night minutes'] + df['total intl minutes']
myTable = df.pivot_table(values = ['total day minutes', 'total eve minutes', 'total night minutes', 'total intl minutes', 'totalMinutes'], index = ['state'], aggfunc = 'sum')
myTable

In [ ]:
myTable.plot.bar();
# plt.rcParams['figure.figsize'] = (50, 30) # раскомментировать для того, чтобы увеличить гравик снизу.

###### Ответ:

In [ ]:
myTable[myTable['totalMinutes'] == myTable['totalMinutes'].max()]

## Жители штата ***WV*** (West Virginia) говорят по телефону больше всех.

In [ ]:
myTable[myTable['totalMinutes'] == myTable['totalMinutes'].min()]

## Жители штата ***CA*** (California) говорят по телефону меньше всех.

Верно ли, что люди, совершающие много международных звонков, приобретают **international plan**? Не забывайте о визуализации информации.

In [ ]:
df.head(6)

In [ ]:
# Your code
internationPlanCalls =  df.pivot_table(values = ['total intl calls'], index = ['international plan'], aggfunc = 'sum')
internationPlanCalls

In [ ]:
internationPlanCalls.plot.bar(color = 'orange');

In [ ]:
middleCalls = df['total intl calls'].mean()
print("Среднее количество звонков за границу: " + str(middleCalls))

In [ ]:
df[df['total intl calls'] > df['total intl calls'].mean()].head(6)

In [ ]:
plt.figure(figsize = (50, 10))
sns.countplot(y = 'total intl calls', hue = 'international plan', data = df[df['total intl calls'] > df['total intl calls'].mean()]);

In [ ]:
internationalPlanMinutes = df.pivot_table(values = ['total intl minutes'], index = ['international plan'], aggfunc = 'sum')
internationalPlanMinutes

In [ ]:
internationalPlanMinutes.plot.bar(color = "orange");

In [ ]:
print("Средняя длина разговоров: " + str(df['total intl minutes'].mean()))

In [ ]:
plt.figure(figsize = (50, 30))
sns.countplot(y = 'total intl minutes', hue = 'international plan', data = df[df['total intl minutes'] > df['total intl minutes'].mean()]);

### **Неверно, что люди, совершающие много международных звонков, приобретают international plan.**

Как связаны наличие голосовой почты и международного плана с лояльностью абонентов? Проверьте статистическую значимость этих связей.

In [ ]:
df.head(6)

#### 1) Связь между голосовой почтой и лояльностью абонентов.

In [ ]:
pd.crosstab(df['voice mail plan'], df['churn'])

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact
chi2_contingency(pd.crosstab(df['voice mail plan'], df['churn']))

In [ ]:
fisher_exact(pd.crosstab(df['voice mail plan'], df['churn']))

#### Видим, что связь между международным планом и лояльностью абонентов есть, причём высокая.

#### 2) Связь между международным планом и лояльностью абонентов.

In [ ]:
pd.crosstab(df['international plan'], df['churn'])

In [ ]:
chi2_contingency(pd.crosstab(df['international plan'], df['churn']))

In [ ]:
fisher_exact(pd.crosstab(df['international plan'], df['churn']))

#### Видим, что связь между международным планом и лояльностью абонентов есть, причём высокая. И больше чем между голосовой почтой и лояльностью абонентов.

Что можно сказать о взаимосвязи количества обращений в колл-центр и лояльности? Не забывайте о визуализации информации.

In [ ]:
# Your code
callCentre = df.pivot_table(values = ['customer service calls'], index = ['churn'], aggfunc = 'sum')
callCentre

In [ ]:
plt.figure(figsize=(40, 15))
sns.countplot(y='customer service calls', hue='churn', data=df);

In [ ]:
from scipy.stats import pointbiserialr
pointbiserialr(df['customer service calls'], df['churn'])

#### Слабая взаимосвязь количества обращений в колл-центр и лояльности присутствует. Чем больше клиент обращался в колл-центр, тем выше вероятность того, что он перестанет пользоваться услугами компании. 

Какие из числовых признаков имеют наибольшую степень корреляции? Почему?

In [ ]:
# Your code
numeric = ['account length', 'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 'total eve minutes', 'total eve calls', 'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 'total intl minutes', 'total intl calls', 'total intl charge', 'customer service calls']
sns.pairplot(df[numeric]);

In [ ]:
sns.heatmap(df[numeric].corr(method='spearman'));

In [ ]:
sns.heatmap(df[numeric].corr(method='pearson'));

In [ ]:
sns.heatmap(df[numeric].corr(method='kendall'));

#### Признаки общего времени звонков и оплата клиента имеют наибольшую степень корреляции, что в принципе естевстенно. Также присутствуют взаимосвязь между количеством звонков и общей оплатой,  количеством звонков и общим временем. Также заметна связь между количеством звонков в разное время. Если клиент достаточно много звонил днём, то есть вероятность, что он звонил и вечером и ночью.

Связано ли время пользования аккаунтом с количеством обращений в колл-центр?

In [ ]:
# Your code
from scipy.stats import pearsonr, spearmanr, kendalltau
pearsonr(df['account length'], df['customer service calls'])

In [ ]:
kendalltau(df['account length'], df['customer service calls'])

#### Коэффициенты корреляций очень малы (почти 0), то есть получается, что связи между временем пользования аккаунтом с количеством обращений в колл-центр никакой нет. Результат статистически не значимый.

Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.

#### Узнать в каком штате больше всего используют голосовые сообщения. А также меньше всего.

In [ ]:
totalMessages = df.pivot_table(values = ['number vmail messages'], index = ['state'], aggfunc = 'sum')
totalMessages.T

In [ ]:
totalMessages[totalMessages['number vmail messages'] == totalMessages['number vmail messages'].max()]

#### В штате *WV* (West Viginia) больше всего используют голосовые сообщения.

In [ ]:
totalMessages[totalMessages['number vmail messages'] == totalMessages['number vmail messages'].min()]

#### В штате *WA* (Washington) меньше всего используют голосовые сообщения.

Верно ли, что чем больше голосовых сообщений, тем больше вероятность, что клиент не покинет компанию?

In [ ]:
df.groupby('churn')['number vmail messages'].count()

In [ ]:
df.groupby('churn')['number vmail messages'].hist();

In [ ]:
pointbiserialr(df['number vmail messages'], df['churn'])

#### Как видим, нет зависимости между количеством голосовых сообщений и оттоком. Коэффициент корреляции слишком малый.

#### В каком штате больше всего "оттока"? В каком меньше всего?

In [ ]:
stateChurn = df.pivot_table(values = ['churn'], index = ['state'], aggfunc = 'sum')
stateChurn.T

In [ ]:
stateChurn.plot.bar(color = 'orange');

In [ ]:
stateChurn[stateChurn['churn'] == stateChurn['churn'].max()].T

#### Видим, что отток больше всего в NJ(New Jersey) и TX(Texas).

In [ ]:
stateChurn[stateChurn['churn'] == stateChurn['churn'].min()].T

#### Видим, что отток меньше всего в AK(Alaska) и HI(Hawaii) и IA(Iowa).

#### Верно ли, что чем дольше клиент пользуется услугами компании, тем больше вероятность, что он не перестанет ими пользоваться?

In [ ]:
df.groupby('churn')['account length'].hist()
plt.ylabel('users');
plt.xlabel('account length');

In [ ]:
pearsonr(df['account length'], df['churn'])

#### Видим, что коэффициент корреляции очень мал, зависимости между длительностью пользования услугами компании и того, что он не перестанет ими пользоваться нет.